# Random Forest
Till now, we only analyzed the Nucleus data.
Now join the RBC data.

Challenge:
CellProfiler wrote nothing for patches with no RBC.
So, the two tables will have different number of rows. 

In [13]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-05-26 10:07:50.446597
Python 3.8.3
sklearn 1.1.1


[]

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [19]:
# Data straight from CellProfiler (rows for every patch image file)
BASE_DIR='//Users/jasonmiller/Downloads/Process100/'
CLASS_DIRS=['0','1','2','3','4','5']
NUM_CLASSES=len(CLASS_DIRS)
IMG_FN="Process100_Image.csv"
RBC_FN="Process100_MergeRBC.csv"
NUC_FN="Process100_Nucleus.csv"
CEL_FN="Process100_ExpandCells.csv"
TIS_FN="Process100_Tissue.csv"
# Saving our models
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.01/'
DESCRIBE=False   # True => stats for every column, False => just mean over objects in patch
SHOW_ALL_COLUMNS=True  

In [20]:
if SHOW_ALL_COLUMNS:
    nuc_df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN)
    initial_cols = nuc_df.columns
    for col in initial_cols:
        print(col)
    nuc_df = None

ImageNumber
ObjectNumber
FileName_Tumor
PathName_Tumor
AreaShape_Area
AreaShape_BoundingBoxArea
AreaShape_BoundingBoxMaximum_X
AreaShape_BoundingBoxMaximum_Y
AreaShape_BoundingBoxMinimum_X
AreaShape_BoundingBoxMinimum_Y
AreaShape_Center_X
AreaShape_Center_Y
AreaShape_CentralMoment_0_0
AreaShape_CentralMoment_0_1
AreaShape_CentralMoment_0_2
AreaShape_CentralMoment_0_3
AreaShape_CentralMoment_1_0
AreaShape_CentralMoment_1_1
AreaShape_CentralMoment_1_2
AreaShape_CentralMoment_1_3
AreaShape_CentralMoment_2_0
AreaShape_CentralMoment_2_1
AreaShape_CentralMoment_2_2
AreaShape_CentralMoment_2_3
AreaShape_Compactness
AreaShape_ConvexArea
AreaShape_Eccentricity
AreaShape_EquivalentDiameter
AreaShape_EulerNumber
AreaShape_Extent
AreaShape_FormFactor
AreaShape_HuMoment_0
AreaShape_HuMoment_1
AreaShape_HuMoment_2
AreaShape_HuMoment_3
AreaShape_HuMoment_4
AreaShape_HuMoment_5
AreaShape_HuMoment_6
AreaShape_InertiaTensorEigenvalues_0
AreaShape_InertiaTensorEigenvalues_1
AreaShape_InertiaTensor_0_0
Ar

In [21]:
if SHOW_ALL_COLUMNS:
    rbc_df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+RBC_FN)
    initial_cols = rbc_df.columns
    for col in initial_cols:
        print(col)
    rbc_df = None

ImageNumber
ObjectNumber
FileName_Tumor
PathName_Tumor
AreaShape_Area
AreaShape_BoundingBoxArea
AreaShape_BoundingBoxMaximum_X
AreaShape_BoundingBoxMaximum_Y
AreaShape_BoundingBoxMinimum_X
AreaShape_BoundingBoxMinimum_Y
AreaShape_Center_X
AreaShape_Center_Y
AreaShape_CentralMoment_0_0
AreaShape_CentralMoment_0_1
AreaShape_CentralMoment_0_2
AreaShape_CentralMoment_0_3
AreaShape_CentralMoment_1_0
AreaShape_CentralMoment_1_1
AreaShape_CentralMoment_1_2
AreaShape_CentralMoment_1_3
AreaShape_CentralMoment_2_0
AreaShape_CentralMoment_2_1
AreaShape_CentralMoment_2_2
AreaShape_CentralMoment_2_3
AreaShape_Compactness
AreaShape_ConvexArea
AreaShape_Eccentricity
AreaShape_EquivalentDiameter
AreaShape_EulerNumber
AreaShape_Extent
AreaShape_FormFactor
AreaShape_HuMoment_0
AreaShape_HuMoment_1
AreaShape_HuMoment_2
AreaShape_HuMoment_3
AreaShape_HuMoment_4
AreaShape_HuMoment_5
AreaShape_HuMoment_6
AreaShape_InertiaTensorEigenvalues_0
AreaShape_InertiaTensorEigenvalues_1
AreaShape_InertiaTensor_0_0
Ar

In [23]:
def get_good_columns():
    GOOD_COLS=['ImageNumber']
    GOOD_COLS.extend(['AreaShape_Area','AreaShape_Compactness','AreaShape_ConvexArea'])
    #GOOD_COLS.extend(['AreaShape_Eccentricity','AreaShape_EquivalentDiameter','AreaShape_EulerNumber'])
    #GOOD_COLS.extend(['AreaShape_Extent','AreaShape_FormFactor','AreaShape_MajorAxisLength'])
    #GOOD_COLS.extend(['AreaShape_MaxFeretDiameter','AreaShape_MaximumRadius','AreaShape_MeanRadius'])
    #GOOD_COLS.extend(['AreaShape_MinFeretDiameter','AreaShape_MinorAxisLength','AreaShape_Orientation'])
    #GOOD_COLS.extend(['AreaShape_Perimeter','AreaShape_Solidity'])
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_CentralMoment') ) ] )
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_HuMoment') ) ] )
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_NormalizedMoment') ) ] )
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_SpacialMoment') ) ] )  # yes
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_Zernike') ) ] )   # yes
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Granularity') ) ] )   # yes!!!
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Intensity') ) ] )   # yes!!!
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Neighbors') ) ] )   
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('RadialDistribution') ) ] )   
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_') ) ] )  # yes
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Contrast_Hematoxylin') ) ] )   
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_DifferenceEntropy_Hematoxylin') ) ] ) 
    #GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('Texture_Entropy_Hematoxylin') ) ] )  # yes
    return GOOD_COLS

In [41]:
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_one_file(fn,cols,prefix):
    set_prefix(prefix)
    object_df = pd.read_csv(fn)
    object_df = object_df[cols]
    object_df.rename(get_prefix,axis='columns',inplace=True)
    object_df.rename(columns={get_prefix('ImageNumber'):'ImageNumber'},inplace=True)
    if DESCRIBE:
        patch_df = object_df.groupby(['ImageNumber']).describe() ## this is slow
        # The random forest classifier cannot handle two-part column names, so convert them to string.    
        #patch_df.columns=patch_df.columns.map(str) # works but leaves in parenthesis
        patch_df.columns=patch_df.columns.map('_'.join)
    else:
        #patch_df = object_df.groupby(['ImageNumber']).mean()
        patch_df = object_df.groupby(['ImageNumber']).max()
    # By virtue of groupby, ImageNumber is now the dataframe index.
    return patch_df


In [42]:
SHOW_DEMO = True
total_df = None
if SHOW_DEMO:
    good_cols = get_good_columns()
    nuc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN,good_cols,'Nuc_')
    rbc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+RBC_FN,good_cols,'Rbc_')
    total_df = nuc_df.join(rbc_df,how='outer')
    # Sources of NaN:
    # CellProfiler reports NaN for stdev of 1 value.
    # CellProfiler omits patches with no RBC; pandas join sets those to NaN.
    total_df.fillna(0,inplace=True)  
total_df

,Nuc_AreaShape_Area,Nuc_AreaShape_Compactness,Nuc_AreaShape_ConvexArea,Rbc_AreaShape_Area,Rbc_AreaShape_Compactness,Rbc_AreaShape_ConvexArea
ImageNumber,,,,,,
1,313,1.273323,345,0.0,0.000000,0.0
2,336,2.208444,365,0.0,0.000000,0.0
3,585,1.701673,603,0.0,0.000000,0.0
4,635,2.727914,818,1696.0,4.213407,2754.0
5,832,2.067581,896,0.0,0.000000,0.0
...,...,...,...,...,...,...
96,340,1.604151,356,0.0,0.000000,0.0
97,812,2.097841,940,428.0,1.346961,448.0
98,633,2.326529,702,0.0,0.000000,0.0


In [44]:
print(datetime.datetime.now())
df = None
def load_all_classes():
    
    good_cols = get_good_columns()
    nuc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN,good_cols,'Nuc_')
    rbc_df = load_one_file(BASE_DIR+CLASS_DIRS[0]+'/'+RBC_FN,good_cols,'Rbc_')
    X = nuc_df.join(rbc_df,how='outer')
    size = len(X)
    y = np.zeros(size)
    for i in range(1,NUM_CLASSES):
        nuc_df = load_one_file(BASE_DIR+CLASS_DIRS[i]+'/'+NUC_FN,good_cols,'Nuc_')
        rbc_df = load_one_file(BASE_DIR+CLASS_DIRS[i]+'/'+RBC_FN,good_cols,'Rbc_')
        Xi = nuc_df.join(rbc_df,how='outer')
        size = len(Xi)
        yi = np.ones(size) * i
        X = pd.concat( (X,Xi) )
        y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
X,y=load_all_classes()
X

2022-05-26 10:38:11.835863


,Nuc_AreaShape_Area,Nuc_AreaShape_Compactness,Nuc_AreaShape_ConvexArea,Rbc_AreaShape_Area,Rbc_AreaShape_Compactness,Rbc_AreaShape_ConvexArea
ImageNumber,,,,,,
1,313,1.273323,345,0.0,0.000000,0.0
2,336,2.208444,365,0.0,0.000000,0.0
3,585,1.701673,603,0.0,0.000000,0.0
4,635,2.727914,818,1696.0,4.213407,2754.0
5,832,2.067581,896,0.0,0.000000,0.0
...,...,...,...,...,...,...
96,1323,3.283374,1474,0.0,0.000000,0.0
97,923,2.000305,1052,306.0,1.367940,342.0
98,1565,2.337659,1833,255.0,2.296032,280.0


In [45]:
print(datetime.datetime.now())
Xtrain,Xvalid,ytrain,yvalid = train_test_split(X, y.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xtrain.shape,'ytrain',ytrain.shape,'non-zero:',np.count_nonzero(ytrain))
print('Xvalid',Xvalid.shape,'yvalid',yvalid.shape,'non-zero:',np.count_nonzero(yvalid))

2022-05-26 10:38:24.700862
Xtrain (450, 6) ytrain (450,) non-zero: 376
Xvalid (150, 6) yvalid (150,) non-zero: 124


In [46]:
# RandomForestClassifier can only track feature names of type string.
num_problems=0
for name in Xtrain.columns:
    if not isinstance(name,str):
        num_problems += 1
        print(type(name),name)
if num_problems==0:
    print("Ok")

Ok


In [47]:
print(datetime.datetime.now())
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model.fit(self.Xtr,self.ytr)
        #print(dir(self.model))  # see whether feature_names_in_ got created
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  # bug fix
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

2022-05-26 10:38:26.777151


In [48]:
print('Train on all Features')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
rf1.set_validation(Xvalid,yvalid)
print(datetime.datetime.now())
rf1.fit()
print(datetime.datetime.now())
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())
print('The impurity-based feature importances.')
top = rf1.important_features()
top.head()

Train on all Features
2022-05-26 10:38:27.723044
2022-05-26 10:38:27.939066
Accuracy: 35.333333333333336
Confusion:
[[11  1  5  3  6  0]
 [ 0 11  0  3  2  3]
 [ 6  5  8  8  1  1]
 [ 7  2  1  4  3  4]
 [ 1  3  4  2  9  4]
 [ 0 10  4  4  4 10]]
The impurity-based feature importances.


,0,1
0,0.283656,Nuc_AreaShape_Area
1,0.267275,Nuc_AreaShape_ConvexArea
2,0.257244,Nuc_AreaShape_Compactness
3,0.067941,Rbc_AreaShape_Compactness
4,0.062044,Rbc_AreaShape_Area


GPU was not active:  
2022-05-25 16:35:28.165143  
2022-05-25 16:35:28.524197

GPU probably not active:  
2022-05-25 16:50:42.142147  
2022-05-25 16:50:42.486454

Why isn't pandas using the GPU?  
Why isn't RF using the GPU?